In [ ]:
import os
os.chdir("..")
from loaders import load_resisc45
from models import CropAndResize, SRResNet, SRGAN
import numpy as np

In [ ]:
dataset, labels = load_resisc45(r"C:\Users\nedst\Desktop\synoptic-project-NedStickler\.venv\Lib\site-packages\tensorflow_datasets")

In [ ]:
batch = dataset[:30]
lr_images, hr_images = CropAndResize(4)(batch)

lr_images = lr_images.numpy().astype(np.uint8)
hr_images = hr_images.numpy().astype(np.uint8)

lr_image = lr_images[0]
hr_image = hr_images[0]

In [ ]:
lr_images.nbytes / 1_000_000

In [ ]:
hr_image.nbytes / 1_000_000

In [ ]:
srresnet = SRResNet(16, 4)
srresnet.summary()

In [ ]:
srgan = SRGAN(srresnet.get_model(), vgg=None)
srgan.summary()